In [29]:
# ## 1 - metrics for the tuned model
# ## 2- oop: 
#     a) Please enter the customer id and target
#     b) if it is out --> sorry you are not in the train data.
#     c) return --> forecasting value with upper and lower bound 31/march/2022
#     d) additional: Ask do you want better forecasting:
#             Yes/No 
#             if no --> thanks you are satsfied (tuned model)
#             if yes --> gor for the difference model 
#             ##hupyter and py file.

In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)   
from sklearn.metrics import mean_squared_error
import pandas as pd
from pycaret.time_series import TSForecastingExperiment
from pycaret.datasets import get_data

import warnings
warnings.filterwarnings('ignore')

df = pd.read_excel('budget_data_updated_new (2).xlsx')

C:\Users\avitr\AppData\Roaming\Python\Python38\site-packages\sktime\utils\validation\series.py:21: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  VALID_INDEX_TYPES = (pd.Int64Index, pd.RangeIndex, pd.PeriodIndex, pd.DatetimeIndex)
C:\Users\avitr\AppData\Roaming\Python\Python38\site-packages\sktime\forecasting\base\_fh.py:18: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  RELATIVE_TYPES = (pd.Int64Index, pd.RangeIndex)
C:\Users\avitr\AppData\Roaming\Python\Python38\site-packages\sktime\forecasting\base\_fh.py:19: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  ABSOLUTE_TYPES = (pd.Int64Index, pd.RangeIndex, pd.DatetimeIndex, pd.PeriodIndex)
C:\Users\avitr\AppData\Roamin

In [87]:
class MyModel:
    
    def __init__(self,df,my_list,target):
        self.outputs = ['Groceries', 'Transportation', 'Health', 'Utilities', 'DiningOut', 'BillsAndServices', 'Shopping', 'Entertainment','MoneyTransfer']
        
        self.df = df
        self.df = self.df.set_index('cif')
        
        self.df_outputs = self.df[self.outputs]
        self.df_outputs = self.df_outputs.mask(self.df_outputs.eq(0), self.df_outputs.groupby('cif').transform('mean'))
        
        self.df = self.df.drop(self.outputs,axis=1)
        self.df_new = pd.concat([self.df,self.df_outputs],axis=1)
        #df_new = df_new.set_index('cif')
        
        self.df = self.df_new.copy()
        self.df_1 = self.df.query('customer_category == 1')
        self.df_1.replace(0,self.df_1[self.outputs].mean(axis=0),inplace=True)

        self.df_2 = self.df.query('customer_category == 2')
        self.df_2.replace(0,self.df_2[self.outputs].mean(axis=0),inplace=True)
        self.df_3 = self.df.query('customer_category == 3')
        self.df_3.replace(0,self.df_3[self.outputs].mean(axis=0),inplace=True)
        self.df_4 = self.df.query('customer_category == 4')
        self.df_4.replace(0,self.df_4[self.outputs].mean(axis=0),inplace=True)
        self.df_5 = self.df.query('customer_category == 5')
        self.df_5.replace(0,self.df_5[self.outputs].mean(axis=0),inplace=True)
        self.df = pd.concat([self.df_1,self.df_2,self.df_3,self.df_4,self.df_5])

        self.my_list = my_list
        self.target = target
        
        self.df['GENDER_DESC'] = self.df['GENDER_DESC'].replace(['Male','Female'],[1,0])
        self.df = pd.get_dummies(self.df, columns=['CUST_SEGMENT_BAL'])
        
        self.cols = ['date','customer_category', 'NO_OF_ALL_CC', 'NO_OF_SAL_HITS','KD_BAL_ALL_ACC',
                'GENDER_DESC','CUST_AGE',  'NO_OF_ALL_ACC', 'IS_CUST_HOLDING_PF', 'IS_CUST_HOLDING_CC', 
                'IS_CUST_HOLDING_PRE_PAID', 'IS_CUST_WITH_SAL_TRF', 'NO_OF_POS_TXS','KD_OF_ALL_POS_TXS',
                'NO_OF_ALL_PRODUCTS', 'NO_OF_ALL_PRODUCT_CATEGORIES','CUST_SEGMENT_BAL_1. Basic Saver',
                'CUST_SEGMENT_BAL_1. Inactive', 'CUST_SEGMENT_BAL_2. Smart Saver', 'CUST_SEGMENT_BAL_3. Long Term Planner',self.target]
        self.df = self.df[self.cols]
        
        self.Filter_df  = self.df[self.df.index.isin(self.my_list)]
        self.Filter_df = self.Filter_df.reset_index()
        self.Filter_df = self.Filter_df.drop(['cif'],axis=1)
        self.Filter_df = self.Filter_df.sort_values(by='date')
        self.Filter_df = self.Filter_df.set_index('date')
        
    def get_tuned_model(self):

        self.selection(self.Filter_df,self.target)
        
    def get_diff_model(self):
        
        df_log=np.sqrt(self.Filter_df[ self.target])
        df_diff=df_log.diff().dropna()
        df_diff = pd.concat([self.Filter_df,df_diff],axis=1)
        df_diff = df_diff.iloc[1: , :]
        y_diff =df_diff.iloc[:,-1]
        df_diff = df_diff.drop([ self.target],axis=1)
        df_diff = pd.concat([df_diff,y_diff],axis=1)
        self.selection(df_diff,self.target)
        
    def selection(self,Filter_df,target):
        FH = 1
        data_for_modeling = Filter_df
        future_data = Filter_df.iloc[-1:]
        future_exog = future_data.drop(columns=target)

#         print(data_for_modeling.shape, future_data.shape, future_exog.shape)

        #### Setup Experiment ----
        exp= TSForecastingExperiment()
        exp.setup(
            data=data_for_modeling, target=target, fh=FH, seasonal_period=1, session_id=42,fold=1
        ,enforce_pi=True)

        #### Baseline Model ----
        model = exp.create_model("auto_arima")
        preds = exp.predict_model(model)
#         exp.plot_model(model)

        #### Tune Model ----
        tuned_model = exp.tune_model(model)
        tuned_preds = exp.predict_model(tuned_model)
#         exp.plot_model(tuned_model)

        #### Finalize Model ----
        final_model = exp.finalize_model(tuned_model)
        future_preds = exp.predict_model(final_model, X=future_exog,return_pred_int=True,alpha =0.3)
        print(future_preds)
#         exp.plot_model(final_model, data_kwargs={'X': future_exog})

In [89]:
if __name__ == "__main__":
    cust_id = int(input('Please Enter a customer id: '))
    target = input('Please enter the target: ')
    if cust_id not in set(df['cif']):
        print('Sorry you are not in the train data.')
    else:
        model = MyModel(df,[cust_id],target)
        model.get_tuned_model()
        print('\n')
        choice = input('DO you need a better model? y/n: ')
        if choice.lower() == 'n':
            print('\nThanks you are satsfied')
        else:
            model.get_diff_model()

Please Enter a customer id: 333
Please enter the target: dd
Sorry you are not in the train data.


In [84]:
# 1208221 Shopping